In [1]:
!pip install transformers

In [2]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from sklearn.metrics import f1_score
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import Dataset, DataLoader

In [3]:
dataset = pd.read_csv('/kaggle/input/no-naives/dataset_all_nlp_features_target_classes_no_naive.csv', sep='\t')
dataset = dataset[dataset.question_id != '129_Чтение текста - видео']
dataset = dataset[dataset['N_words'] > 2]
# dataset = dataset[dataset["verb"] > 0]
# dataset = dataset[dataset["raven"] > 0]

In [4]:
dataset.describe()

,task_id,user_id,age,E,A,C,N,O,AP,AM,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
count,6477.000000,6477.000000,1736.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,...,6467.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000,6477.000000
mean,6.098966,658.156091,23.868664,3.520113,3.681306,3.681509,2.960723,3.755056,3.840862,2.150763,...,1.666306,2.035510,0.516134,1.232978,0.844527,0.920179,0.710514,1.008492,0.695538,1.330400
std,3.536441,279.490692,6.199752,0.827778,0.678065,0.826531,0.894088,0.653033,0.551309,0.681014,...,1.077559,1.333504,0.725172,0.762120,0.810527,0.759715,0.781998,0.772894,0.826484,0.725106
min,1.000000,124.000000,18.000000,1.130000,1.444444,1.330000,1.000000,1.700000,1.670000,1.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,437.000000,20.000000,2.880000,3.222222,3.000000,2.250000,3.300000,3.555556,1.666667,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,6.000000,654.000000,22.000000,3.500000,3.670000,3.777778,3.000000,3.800000,3.888889,2.110000,...,2.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,9.000000,874.000000,25.000000,4.250000,4.220000,4.333333,3.625000,4.300000,4.220000,2.555556,...,2.000000,3.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000
max,12.000000,1167.000000,51.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,4.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [13]:
def get_circumplex_labels(dataset):
    AP_LABELS = list(set(dataset['AP_classes'].unique()))
    AP_SCORE_INDICES = range(0, 3)
    AM_LABELS = list(set(dataset['AM_classes'].unique()))
    AM_SCORE_INDICES = range(3, 6)

    BP_LABELS = list(set(dataset['BP_classes'].unique()))
    BP_SCORE_INDICES = range(6, 9)
    BM_LABELS = list(set(dataset['BM_classes'].unique()))
    BM_SCORE_INDICES = range(9, 12)

    GP_LABELS = list(set(dataset['GP_classes'].unique()))
    GP_SCORE_INDICES = range(12, 15)
    GM_LABELS = list(set(dataset['GM_classes'].unique()))
    GM_SCORE_INDICES = range(15, 18)

    DP_LABELS = list(set(dataset['DP_classes'].unique()))
    DP_SCORE_INDICES = range(18, 21)
    DM_LABELS = list(set(dataset['DM_classes'].unique()))
    DM_SCORE_INDICES = range(21, 24)

    CIRCLE_LABELS = AP_LABELS + AM_LABELS + BP_LABELS + BM_LABELS + GP_LABELS + GM_LABELS + DP_LABELS + DM_LABELS
    INDEXEX_CIRCLE = [AP_SCORE_INDICES, AM_SCORE_INDICES, BP_SCORE_INDICES,
                      BM_SCORE_INDICES, GP_SCORE_INDICES, GM_SCORE_INDICES,
                      DP_SCORE_INDICES, DM_SCORE_INDICES]
    return INDEXEX_CIRCLE, CIRCLE_LABELS


In [14]:
curc_indexes, curc_labels = get_circumplex_labels(dataset)

In [15]:
curc_indexes

[range(0, 3),
 range(3, 6),
 range(6, 9),
 range(9, 12),
 range(12, 15),
 range(15, 18),
 range(18, 21),
 range(21, 24)]

In [17]:
class DataPreparation(Dataset):
    
    def __init__(self, tokenizer, data, target_values='circumplex', max_length=None):
        
        self.tokenizer = tokenizer
        self.data = data
        self.t_val = target_values
        
        if max_length == None:
            max_length_counted = data["text"].str.split(' ').str.len().max(axis=0)
            self.max_length = max_length_counted if max_length_counted < 512 else 512
        else:
            self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def tokenize(self, text):

        tokens = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt')

        return tokens
    

    def reorganize_labels(self, row):

        def return_zeros(row_data, classes):
          labels_zeros = np.zeros(len(set(self.data[classes].unique())))
          labels_zeros[row_data[classes]] = 1
          return labels_zeros

        return np.concatenate((return_zeros(row, 'AP_classes'), return_zeros(row, 'AM_classes'),
                               return_zeros(row, 'BP_classes'), return_zeros(row, 'BM_classes'),
                               return_zeros(row, 'GP_classes'), return_zeros(row, 'GM_classes'),
                               return_zeros(row, 'DP_classes'), return_zeros(row, 'DM_classes')
                               ))   


    def __getitem__(self, index):
        
        source_text = self.data['text'].iloc[index]
        source = self.tokenize(source_text)

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        
        labels = self.reorganize_labels(self.data.iloc[index])
        

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "labels":  labels
        }

In [18]:
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
np.random.seed(112)
train_data, vaild_data, test_data = np.split(dataset.sample(frac=1, random_state=42), 
                                     [int(.80*len(dataset)), int(.91*len(dataset))])

print(len(train_data),len(vaild_data), len(test_data))

5181 713 583


In [20]:
train_data.head(2)

,text,f,filename,task_id,question_id,user_id,gender,age,E,A,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
2138,стала лучшим консультантом памперс,NaN,аудио/answer11_113_12665_3_241.wav,11,134_Видео-интервью,241,NaN,NaN,4.13,3.330000,...,0.0,0,0,2,2,1,1,1,2,0
2467,Вы знаете я нашла себе работу по любимую работу,NaN,аудио/answer11_113_20670_6_782.wav,11,134_Видео-интервью,782,NaN,NaN,4.00,3.555556,...,4.0,4,1,0,0,0,0,0,0,1


In [21]:
train_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = train_data,
    max_length = 120,
)

val_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = vaild_data,
    max_length = 120,
)

test_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = test_data,
    max_length = 120,
)

In [22]:
train_dataset = DataLoader(train_dataset_data, batch_size=8, drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset_data, batch_size=8)
test_dataset = DataLoader(test_dataset_data, batch_size=8)

In [24]:
b = next(iter(train_dataset))
for k, v in b.items():
    print(f'{k} shape: {v.shape}')
    # print(b['labels'])

source_ids shape: torch.Size([8, 120])
source_mask shape: torch.Size([8, 120])
labels shape: torch.Size([8, 24])


In [25]:
class RNN_Block(nn.Module):
    
    def __init__(self, input_size=768, hidden_size=512, rnn='LSTM', biderectional=True):
        super().__init__()

        rnn_type = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}
        self.rnn = rnn_type[rnn](input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=biderectional)


    def forward(self, input_ids):
        
        rnn_output = self.rnn(input_ids)
        return rnn_output


class CNN_Block(nn.Module):
    
    def __init__(self, input_size=1024, out_size=64, kernel_size=3, stride=1, padding=1):
        super().__init__()

        self.conv_2 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=2, stride=stride, padding=padding)
        self.conv_3 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=3, stride=stride, padding=padding)
        self.conv_5 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=5, stride=stride, padding=padding+1)
        self.relu = nn.ReLU()


    def forward(self, sequence_input):

        conv_input = sequence_input.permute(0, 2, 1) # batch_size, hidden_size, sequence_length
        cnn_output2 = self.conv_2(conv_input)
        cnn_output2 = self.relu(cnn_output2)
        cnn_output2 = F.max_pool1d(cnn_output2, kernel_size=cnn_output2.shape[2])
        cnn_output3 = self.conv_3(conv_input)
        cnn_output3 = self.relu(cnn_output3)
        cnn_output3 = F.max_pool1d(cnn_output3, kernel_size=cnn_output3.shape[2])
        cnn_output5 = self.conv_5(conv_input)
        cnn_output5 = self.relu(cnn_output5)
        cnn_output5 = F.max_pool1d(cnn_output5, kernel_size=cnn_output5.shape[2])
        cnn_output = torch.cat([cnn_output2.squeeze(dim=2), cnn_output3.squeeze(dim=2), cnn_output5.squeeze(dim=2)], dim=1)

        return cnn_output


class BIGRU_BILSTM_CNN(nn.Module):
    
    def __init__(self, out_features, hidden_size=512, hidden_size_lin=128,
                 p_spatial_dropout=0.5, out_chanels_cnn=64, kernel_size_cnn=3, 
                 stride_cnn=1, padding_cnn=1, 
                 rnn_type='LSTM', biderectional=True, pre_trained='DeepPavlov/rubert-base-cased-sentence'):
        super(BIGRU_BILSTM_CNN, self).__init__()

        self.bert = AutoModel.from_pretrained(pre_trained)
        self.layer_norm = nn.LayerNorm(32)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.p_spatial_dropout = p_spatial_dropout
        self.bigru_block = RNN_Block(input_size=self.bert.config.hidden_size, hidden_size=hidden_size, rnn='GRU', biderectional=True) 
        self.bilstm_block = RNN_Block(input_size=self.bert.config.hidden_size, hidden_size=hidden_size, rnn='LSTM', biderectional=True) 
        
        if biderectional == True:
            self.cnn_block = CNN_Block(input_size=hidden_size*2, out_size=out_chanels_cnn,
                                      kernel_size=kernel_size_cnn, stride=stride_cnn, padding=padding_cnn)
        else:
            self.cnn_block = CNN_Block(input_size=hidden_size, out_size=out_chanels_cnn,
                                      kernel_size=kernel_size_cnn, stride=stride_cnn, padding=padding_cnn)
            

        self.linear_modules = nn.ModuleList([torch.nn.Linear(out_chanels_cnn*3*2, 32),
                                          torch.nn.Linear(32, 32),
                                          torch.nn.Linear(32, out_features)])
                                          
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def forward(self, input_ids, attention_mask):

        # _, cls_hs = self.bert(input_ids, attention_mask = attention_mask, return_dict = False)

        encoded_layers = self.bert(input_ids=input_ids,attention_mask=attention_mask, output_hidden_states=True)
#         encoded_layers = encoded_layers['last_hidden_state'].permute(1, 0, 2)
        x = encoded_layers['last_hidden_state']#.permute(1, 0, 2)
#         # spatial dropout

        embeddings = x.unsqueeze(2)    # (N, T, 1, K)
        embeddings = embeddings.permute(0, 3, 2, 1)  # (N, K, 1, T)
        embeddings = F.dropout2d(embeddings, self.p_spatial_dropout)  # (N, K, 1, T), some features are masked
        embeddings = embeddings.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = embeddings.squeeze(2)  # (N, T, K)

        bigru,_ = self.bigru_block(x) #(N,L,H in) - batch size, sequence length, input size        
        bilstm, hidden_bilstm = self.bilstm_block(x)

        bilstm_cnn = self.cnn_block(bilstm)
        bigru_cnn = self.cnn_block(bigru)
        x = torch.cat([bigru_cnn, bilstm_cnn], dim=1)

        h = []
        
        for lin in self.linear_modules[:-1]:
            x = lin(x)
            h.append(x)
            x = self.relu(x)
            x = self.dropout(x)
        
        x = self.relu(h[-1] + h[-2])
        x = self.linear_modules[-1](x)
        return x

In [27]:
model = BIGRU_BILSTM_CNN(pre_trained=model_name,  out_features=len(curc_labels))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
epochs = 15
model = model.to(device)

In [29]:
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Надо сделать прогон на каждом классе отдельно, чтобы определеить, какие будут веса можно будет использовать для общего лосса

In [30]:
def analysing_logits(logits, class_indexes):
  
  def compute_maxes(logits, CUR_SCORE_INDEX):
    labels_zeros = np.zeros(logits[:, CUR_SCORE_INDEX].shape)
    pred_class = np.argmax(logits[:, CUR_SCORE_INDEX], axis=1)
    labels_zeros[list(range(len(labels_zeros))), pred_class] = 1
    return labels_zeros

  list_of_best_labels = []

  for circ_cl in class_indexes:
    labels_zeros = compute_maxes(logits, circ_cl)
    list_of_best_labels.append(labels_zeros)
  
  return np.concatenate((list_of_best_labels), axis=1)

In [31]:
def init_adding_weights():
    return (1/1.09, 1/1.09, 1/1.09, 1/1.09,
            1/1.09, 1/1.09, 1/1.09, 1/1.09)

In [33]:
def comupte_custom_loss(labels, logits, criterion, class_indexes):
    

    custom_weights = init_adding_weights()

    custom_loss = 0
    for ind, circ_cl in enumerate(class_indexes):

      custom_loss += custom_weights[ind] * criterion(logits[:, circ_cl],
                                  labels[:, circ_cl])
      # print(custom_loss)

    return custom_loss

In [34]:
def show_metrics(labels, predicted, class_indexes):

    labels_names = ['AP', 'AM', 'BP', 'BM',
              'GP', 'GM', 'DP', 'DM']
    
    labels = np.vstack(labels)
    predicted = np.vstack(predicted)

    for lab, circ_cl in zip(labels_names, class_indexes):
      
      f1_sc = f1_score(labels[:, circ_cl],
               predicted[:, circ_cl], average="macro")
      accur = accuracy_score(labels[:, circ_cl],
               predicted[:, circ_cl])
      
      
      print(f'Scores for {lab}\n==================\n')
      print('Macro F1 score for {0} - {1}\n'.format(lab, f1_sc))
      print('Accuracy score for {0} - {1}\n'.format(lab, accur))
      print(classification_report(labels[:, circ_cl], predicted[:, circ_cl], zero_division=0))


In [35]:
def train(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    print('Epoch #{}\n'.format(n_epoch+1))

    train_losses = []
    train_labels = []
    train_predictions = []   

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size), 
                        desc='Epoch {}'.format(n_epoch + 1))

    model.train()

    for _, data in enumerate(data_loader, 0):


          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          optimizer.zero_grad()

          pred = model(input_ids=input_ids, attention_mask=attention_mask)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)

              
          loss.backward()
              
          optimizer.step()
          lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
          scheduler.step()
        
          predict = torch.log_softmax(pred, dim=1)

          resulting_output = analysing_logits(predict.detach().cpu().numpy(), class_indexes)


          train_losses.append(loss.item())
          train_labels.extend(labels.cpu().detach().numpy())
          train_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(train_losses))
          progress_bar.update(1)

    progress_bar.close()
  
    
    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(train_losses)))
    print(f'\n Scores after {n_epoch + 1} on train:')
    show_metrics(train_labels, train_predictions, class_indexes)

In [36]:
def validating(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size),
                        desc='Epoch {}'.format(n_epoch + 1))

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)
        
          predict = torch.log_softmax(pred, dim=1)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)
          
          resulting_output = analysing_logits(predict.detach().cpu().numpy(), class_indexes)

          val_losses.append(loss.item())
          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(val_losses))
          progress_bar.update(1)

    progress_bar.close()
    
    
    valid_stats.append(
        {
            'Val Loss': np.mean(val_losses)
        }
    )

    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(val_losses)))
    print(f'\n Scores after {n_epoch + 1} on validation:')
    show_metrics(val_labels, val_predictions, class_indexes)
    return valid_stats

In [41]:
import os
from transformers import get_linear_schedule_with_warmup, AdamW

In [42]:
valid_stats = []
lr=2e-4
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)
global lr_list
lr_list = []
global scheduler
total_steps = len(train_dataset) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=int(total_steps * 0.1),
                                                num_training_steps=total_steps)

# for each epoch
for epoch in range(epochs):
    # train

    try:
        train(model, train_dataset, device, optimizer, criterion, epoch, curc_indexes)
          # # validate
        
        valid_stats = validating(model, val_dataset, device, optimizer, criterion, epoch, curc_indexes )

        if valid_stats[epoch]['Val Loss'] < best_valid_loss:
            print(f"Best validation loss - {valid_stats[epoch]['Val Loss']}")
            best_valid_loss = valid_stats[epoch]['Val Loss']
            
            name_to_save = f'model_baseline_basic_all_circ'
            if os.path.isfile('results/'+name_to_save+'.pth'):
                    os.remove('results/'+name_to_save+'.pth')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
            else:
                    if not os.path.isdir('results'):
                        os.mkdir('results')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
#                     else:
#                         os.mkdir('results')
    except KeyboardInterrupt:
        break

Epoch #1



Epoch 1:   0%|          | 0/647 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 7.705569319019107

 Scores after 1 on train:
Scores for AP

Macro F1 score for AP - 0.2882590930018416

Accuracy score for AP - 0.4385625965996909

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1056
           1       0.41      0.22      0.28      1896
           2       0.45      0.84      0.58      2224

   micro avg       0.44      0.44      0.44      5176
   macro avg       0.29      0.35      0.29      5176
weighted avg       0.34      0.44      0.35      5176
 samples avg       0.44      0.44      0.44      5176

Scores for AM

Macro F1 score for AM - 0.30379256202890426

Accuracy score for AM - 0.5753477588871716

              precision    recall  f1-score   support

           0       0.62      0.88      0.73      3185
           1       0.27      0.13      0.18      1267
           2       0.18      0.00      0.01       724

   micro avg       0.58      0.58      0.58      5176
   macro avg

Epoch 1:   0%|          | 0/89 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 7.443074237376785

 Scores after 1 on validation:
Scores for AP

Macro F1 score for AP - 0.3329967544175983

Accuracy score for AP - 0.4586255259467041

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       131
           1       0.38      0.51      0.44       253
           2       0.53      0.60      0.56       329

   micro avg       0.46      0.46      0.46       713
   macro avg       0.30      0.37      0.33       713
weighted avg       0.38      0.46      0.41       713
 samples avg       0.46      0.46      0.46       713

Scores for AM

Macro F1 score for AM - 0.2628150665533843

Accuracy score for AM - 0.6507713884992987

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       464
           1       0.00      0.00      0.00       166
           2       0.00      0.00      0.00        83

   micro avg       0.65      0.65      0.65       713
   macro

...



Mean Loss after epoch #14 - 7.352932212279992

 Scores after 14 on validation:
Scores for AP

Macro F1 score for AP - 0.3757323685811332

Accuracy score for AP - 0.4684431977559607

              precision    recall  f1-score   support

           0       0.26      0.08      0.12       131
           1       0.41      0.45      0.43       253
           2       0.53      0.64      0.58       329

   micro avg       0.47      0.47      0.47       713
   macro avg       0.40      0.39      0.38       713
weighted avg       0.44      0.47      0.44       713
 samples avg       0.47      0.47      0.47       713

Scores for AM

Macro F1 score for AM - 0.29246886562782887

Accuracy score for AM - 0.6507713884992987

              precision    recall  f1-score   support

           0       0.66      0.99      0.79       464
           1       0.50      0.01      0.02       166
           2       0.33      0.04      0.07        83

   micro avg       0.65      0.65      0.65       713
   ma

Epoch 15:   0%|          | 0/647 [00:00<?, ?it/s]



Mean Loss after epoch #15 - 7.070600668324999

 Scores after 15 on train:
Scores for AP

Macro F1 score for AP - 0.4399002818796154

Accuracy score for AP - 0.5193199381761978

              precision    recall  f1-score   support

           0       0.45      0.13      0.20      1056
           1       0.47      0.53      0.50      1895
           2       0.57      0.70      0.63      2225

   micro avg       0.52      0.52      0.52      5176
   macro avg       0.49      0.45      0.44      5176
weighted avg       0.51      0.52      0.49      5176
 samples avg       0.52      0.52      0.52      5176

Scores for AM

Macro F1 score for AM - 0.2884391482923736

Accuracy score for AM - 0.6192040185471407

              precision    recall  f1-score   support

           0       0.62      0.99      0.76      3184
           1       0.70      0.01      0.01      1268
           2       0.49      0.05      0.09       724

   micro avg       0.62      0.62      0.62      5176
   macro av

Epoch 15:   0%|          | 0/89 [00:00<?, ?it/s]



Mean Loss after epoch #15 - 7.356078927129863

 Scores after 15 on validation:
Scores for AP

Macro F1 score for AP - 0.391371351464254

Accuracy score for AP - 0.49368863955119213

              precision    recall  f1-score   support

           0       0.31      0.07      0.11       131
           1       0.42      0.49      0.45       253
           2       0.56      0.67      0.61       329

   micro avg       0.49      0.49      0.49       713
   macro avg       0.43      0.41      0.39       713
weighted avg       0.47      0.49      0.46       713
 samples avg       0.49      0.49      0.49       713

Scores for AM

Macro F1 score for AM - 0.2828983256132176

Accuracy score for AM - 0.6437587657784011

              precision    recall  f1-score   support

           0       0.65      0.98      0.78       464
           1       0.00      0.00      0.00       166
           2       0.30      0.04      0.06        83

   micro avg       0.64      0.64      0.64       713
   mac

In [43]:
def test(data_loader, device, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []
    model.load_state_dict(torch.load('results/model_baseline_basic_all_circ.pth'))
    

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)
          predict = torch.log_softmax(pred, dim=1)

          
          resulting_output = analysing_logits(predict.detach().cpu().numpy(), class_indexes)

          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)



    show_metrics(val_labels, val_predictions, class_indexes)
    # print('F1 score after epoch #{0} on validation - {1}\n'.format(str(n_epoch + 1), f1_score(val_labels, val_predictions, average='macro'))

In [44]:
test(train_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.43920387894783414

Accuracy score for AP - 0.5139103554868625

              precision    recall  f1-score   support

           0       0.45      0.16      0.24      1055
           1       0.48      0.43      0.45      1895
           2       0.54      0.76      0.63      2226

   micro avg       0.51      0.51      0.51      5176
   macro avg       0.49      0.45      0.44      5176
weighted avg       0.50      0.51      0.48      5176
 samples avg       0.51      0.51      0.51      5176

Scores for AM

Macro F1 score for AM - 0.2638610253314944

Accuracy score for AM - 0.6170788253477589

              precision    recall  f1-score   support

           0       0.62      1.00      0.76      3186
           1       0.50      0.00      0.00      1267
           2       0.62      0.01      0.03       723

   micro avg       0.62      0.62      0.62      5176
   macro avg       0.58      0.34      0.26      5176
weighted avg       0.59      0.6

In [45]:
test(val_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.3843847784348004

Accuracy score for AP - 0.4796633941093969

              precision    recall  f1-score   support

           0       0.30      0.10      0.15       131
           1       0.43      0.38      0.40       253
           2       0.52      0.71      0.60       329

   micro avg       0.48      0.48      0.48       713
   macro avg       0.42      0.40      0.38       713
weighted avg       0.45      0.48      0.45       713
 samples avg       0.48      0.48      0.48       713

Scores for AM

Macro F1 score for AM - 0.2628150665533843

Accuracy score for AM - 0.6507713884992987

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       464
           1       0.00      0.00      0.00       166
           2       0.00      0.00      0.00        83

   micro avg       0.65      0.65      0.65       713
   macro avg       0.22      0.33      0.26       713
weighted avg       0.42      0.65

In [46]:
test(test_dataset, device, curc_indexes)

Scores for AP

Macro F1 score for AP - 0.4058067282189037

Accuracy score for AP - 0.4854202401372213

              precision    recall  f1-score   support

           0       0.47      0.13      0.21       114
           1       0.45      0.39      0.42       214
           2       0.50      0.73      0.59       255

   micro avg       0.49      0.49      0.49       583
   macro avg       0.47      0.41      0.41       583
weighted avg       0.48      0.49      0.45       583
 samples avg       0.49      0.49      0.49       583

Scores for AM

Macro F1 score for AM - 0.26942027794410595

Accuracy score for AM - 0.6466552315608919

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       376
           1       0.00      0.00      0.00       120
           2       0.50      0.01      0.02        87

   micro avg       0.65      0.65      0.65       583
   macro avg       0.38      0.34      0.27       583
weighted avg       0.49      0.6